In [13]:
import math
import numexpr
import streamlit as st
from langchain_groq import ChatGroq

from langchain_classic.chains.llm import LLMChain
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.callbacks import StreamlitCallbackHandler
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver

In [2]:
@tool
def calculate(expression: str) -> str:
    """Evaluates a mathematical expression safely using numexpr. 
    Use this tool for all math-related questions."""
    try:
        # Define allowed functions for security
        local_dict = {"pi": math.pi, "e": math.e}
        result = numexpr.evaluate(
            expression, global_dict={}, local_dict=local_dict
        )
        return str(result)
    except Exception as e:
        return f"Calculation Error: {e}"
    


In [3]:
## -------setup  Streamlit UI -----
st.set_page_config(page_title="LangChain: Text To Math Problem Solver And Data Search Assistant", page_icon="🦜")
st.title("🦜LangChain: Text To Math Problem Solver And Data Search Assistant.")

2025-11-03 09:05:18.946 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 09:05:18.947 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 09:05:19.032 
  command:

    streamlit run d:\Data_science\Gen AI\langchain_l\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-03 09:05:19.033 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 09:05:19.036 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [4]:
# --- 1. Get the GROQ API Key and URL ---

# Key is read as an empty string on first load
groq_api_key1 = st.text_input("GROQ API key", type="password")
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq


# Load environment variables (like GROQ_API_KEY) from .env file
load_dotenv()



groq_api_key = os.getenv("GROQ_API_KEY")

2025-11-03 10:10:13.458 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 10:10:13.459 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 10:10:13.460 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 10:10:13.460 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 10:10:13.462 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 10:10:13.463 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 10:10:13.464 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [5]:
if not groq_api_key:
    st.info("Please add your GROQ API Key to be continue")
    st.stop()

2025-11-03 09:05:19.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 09:05:19.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 09:05:19.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 09:05:19.077 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [6]:
llm = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=groq_api_key)

In [7]:
# 1. Define the Pydantic input schema
class WikipediaInput(BaseModel):
    """Input for the Wikipedia tool."""
    query: str = Field(description="The search query or topic to look up on Wikipedia.")
    
## Initializing the tools
wikipedia_wrapper = WikipediaAPIWrapper()
wikipedia_tool = StructuredTool(
    name='Wikipedia',
    func=wikipedia_wrapper.run,
    description="A tool for search the internet to find the various information on the topics mentioned",
    args_schema=WikipediaInput

)

In [8]:
# **THIS MUST BE A SIMPLE STRING**
SYSTEM_INSTRUCTIONS = "You are a helpful assistant. Use the calculate tool for math and the wikipedia_tool for facts."


In [9]:
# Optional: Add a checkpointer for state management (memory/history)
memory = MemorySaver()

In [10]:
# Combine all tools into a single, flat list
ALL_TOOLS = [calculate, wikipedia_tool]

In [11]:
## Initialization the Math tools
agent_executor = create_agent(
    model=llm, 
    tools=ALL_TOOLS, 
    system_prompt=SYSTEM_INSTRUCTIONS,
    # Adding the checkpointer for persistent state/history
    checkpointer=memory,
    
)

In [12]:
# --- 4. Run the Agent ---
print("--- Running Fixed Agent ---")
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What is the volume of a sphere with a radius of 5, using 4/3 * pi * r^3, and what is the largest planet?")]},
    config={"configurable": {"thread_id": "math-and-fact-query-fixed"}}
)

print("\n--- Final Answer ---")
print(response["messages"][-1].content)

--- Running Fixed Agent ---

--- Final Answer ---
The largest planet in the solar system is Jupiter.


In [16]:

### Prompt Template 
prompt= ChatPromptTemplate.from_messages(
    [
        ("system", "Your agent tasked for solving user mathemetic  question. logically arrive at the solution  and provide detailed explanation and display it point wise for the question bellow."),
        ("user", "Question:{question}")
    ]
)



In [ ]:
# Creating chain
chain = prompt|llm

In [ ]:


@tool
def reasoning_tool(question: str) -> str:
    """
    A tool for answering logic-based and reasoning questions. 
    Input should be the complete, logic-based question.
    """
    # Use .invoke() for the modern LCEL chain instead of legacy .run()
    return chain.invoke({"question": question}) 
    # NOTE: Adjust the input key ("topic") to match what your 'chain' expects.



In [ ]:
# Combine all tools into a single, flat list
ALL_TOOLS = [calculate, wikipedia_tool, reasoning_tool]

In [30]:
## Initialization the Math tools
agent_executor1 = create_agent(
    model=llm, 
    tools=ALL_TOOLS, 
    system_prompt=SYSTEM_INSTRUCTIONS,
    # Adding the checkpointer for persistent state/history
    checkpointer=memory,
    
)

In [33]:
# --- 4. Run the Agent ---
print("--- Running Fixed Agent ---")
response = agent_executor1.invoke(
    {"messages": [HumanMessage(content="9 people can do a work in 16 days. How many days will 12 people take to do the same work? ")]},
    config={"configurable": {"thread_id": "math-and-fact-query-fixed"}}
)

print("\n--- Final Answer ---")
print(response["messages"][-1].content)

--- Running Fixed Agent ---

--- Final Answer ---
So, 12 people will take 12 days to do the same work.
